# Daniel Lee - FINAL Capstone Project

## i) Importing Libraries

In [5]:
#Import Library
import requests
from bs4 import BeautifulSoup 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import random # library for random number generation

#Installing Folium
!conda install -c conda-forge folium=0.5.0 --yes
import folium
from mpl_toolkits.basemap import Basemap

import json
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans


# Matplotlib and numpy
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt

import numpy as np

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

print('Folium installed and imported!')
print("Import complete!")

# from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

# All requested packages already installed.

Folium installed and imported!
Import complete!


## ii) Setting up Foursquare Credentials

In [57]:
CLIENT_ID = 'O1EG5HDYZEETZIGRG5LSESCSH113KLS1M0ITP3VRU14ZTWUT' # your Foursquare ID
CLIENT_SECRET = 'FP00VEYKKEUX01QWVL3O351XDMZCFXZ1CESEPIREXRBME1CN' # your Foursquare Secret
VERSION = '20181101' # Foursquare API version
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: O1EG5HDYZEETZIGRG5LSESCSH113KLS1M0ITP3VRU14ZTWUT
CLIENT_SECRET:FP00VEYKKEUX01QWVL3O351XDMZCFXZ1CESEPIREXRBME1CN


## iii) Importing the UK Postcodes 
##### Source: https://www.doogal.co.uk/london_postcodes.php

In [17]:
Filename = "https://www.doogal.co.uk/UKPostcodesCSV.ashx?area=London"



In [18]:
df = pd.read_csv(Filename)
print("Done")

Done


In [19]:
#Quick Look at Data
df.head()

,Postcode,In Use?,Latitude,Longitude,Easting,Northing,Grid Ref,County,District,Ward,District Code,Ward Code,Country,County Code,Constituency,Introduced,Terminated,Parish,National Park,Population,Households,Built up area,Built up sub-division,Lower layer super output area,Rural/urban,Region,Altitude,London zone,LSOA Code,Local authority,MSOA Code,Middle layer super output area,Parish Code,Census output area,Constituency Code,Index of Multiple Deprivation,Quality,User Type,Last updated,Nearest station,Distance to station
0,BR1 1AA,Yes,51.401546,0.015415,540291,168873,TQ402688,Greater London,Bromley,Bromley Town,E09000006,E05000109,England,E11000009,Bromley and Chislehurst,2016-05-01,NaN,"Bromley, unparished area",NaN,NaN,NaN,Greater London,Bromley,Bromley 018B,Urban major conurbation,London,71,5,E01000675,NaN,E02000144,Bromley 018,E43000196,E00003264,E14000604,20532,1,0,2018-11-15,Bromley South,0.218257
1,BR1 1AB,Yes,51.406333,0.015208,540262,169405,TQ402694,Greater London,Bromley,Bromley Town,E09000006,E05000109,England,E11000009,Bromley and Chislehurst,2012-03-01,NaN,"Bromley, unparished area",NaN,NaN,NaN,Greater London,Bromley,Bromley 008B,Urban major conurbation,London,71,4,E01000676,NaN,E02000134,Bromley 008,E43000196,E00003255,E14000604,10169,1,0,2018-11-15,Bromley North,0.253666
2,BR1 1AD,No,51.400057,0.016715,540386,168710,TQ403687,Greater London,Bromley,Bromley Town,E09000006,E05000109,England,E11000009,Bromley and Chislehurst,2014-09-01,2017-09-01,"Bromley, unparished area",NaN,NaN,NaN,Greater London,Bromley,Bromley 018B,Urban major conurbation,London,53,5,E01000675,NaN,E02000144,Bromley 018,E43000196,E00003264,E14000604,20532,1,1,2018-11-15,Bromley South,0.044559
3,BR1 1AE,Yes,51.404543,0.014195,540197,169204,TQ401692,Greater London,Bromley,Bromley Town,E09000006,E05000109,England,E11000009,Bromley and Chislehurst,2008-08-01,NaN,"Bromley, unparished area",NaN,34.0,21.0,Greater London,Bromley,Bromley 018C,Urban major conurbation,London,71,4,E01000677,NaN,E02000144,Bromley 018,E43000196,E00003266,E14000604,19350,1,0,2018-11-15,Bromley North,0.462939
4,BR1 1AF,Yes,51.401392,0.014948,540259,168855,TQ402688,Greater London,Bromley,Bromley Town,E09000006,E05000109,England,E11000009,Bromley and Chislehurst,2015-05-01,NaN,"Bromley, unparished area",NaN,NaN,NaN,Greater London,Bromley,Bromley 018B,Urban major conurbation,London,58,5,E01000675,NaN,E02000144,Bromley 018,E43000196,E00003264,E14000604,20532,1,0,2018-11-15,Bromley South,0.227664


#### Selecting Relevant Necessary Columns

In [48]:
PostCode = df[['Postcode','Latitude','Longitude', 'London zone', 'Distance to station', 'District']]
PostCode.columns = ['Postcode','Latitude','Longitude', 'London zone', 'Distance to station', 'Area']
PostCode.head()

,Postcode,Latitude,Longitude,London zone,Distance to station,Area
0,BR1 1AA,51.401546,0.015415,5,0.218257,Bromley
1,BR1 1AB,51.406333,0.015208,4,0.253666,Bromley
2,BR1 1AD,51.400057,0.016715,5,0.044559,Bromley
3,BR1 1AE,51.404543,0.014195,4,0.462939,Bromley
4,BR1 1AF,51.401392,0.014948,5,0.227664,Bromley


In [49]:
# Group longitude and latitude 
df_areas_grouped = PostCode.groupby(PostCode['Area'])

aggregator = {'Latitude':'mean', 'Longitude':'mean'}

df_London_areas = df_areas_grouped.agg(aggregator)

# Reformat the table
df_London_areas = df_London_areas.reset_index()

# Convert to Dictionary
dict_London_areas = df_London_areas.transpose().to_dict()

In [50]:
df_London_areas.head()

,Area,Latitude,Longitude
0,Barking and Dagenham,51.546530,0.125061
1,Barnet,51.605584,-0.207618
2,Bexley,51.459158,0.136218
3,Brent,51.551872,-0.257565
4,Bromley,51.391818,0.026435


## Part 1 - Exploratory Data Analysis using Foursquare

#### a) Explore the Number of gyms in SOHO *(within a 2km radius)
> *Represents Potential Demand of Sports or Exercise in Area*

In [51]:
latitude = 51.5136
longitude = 0.1365
print(latitude, longitude)
# 51.5054° N, 0.0235° W -- Canary Wharf
# 51.5136° N, 0.1365° W is SOHO London

51.5054 0.0235


In [109]:
search_query = 'Gym'
radius = 2000
print("Gyms within 2 km of SOHO")

Gyms within 2 km of SOHO


In [110]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=O1EG5HDYZEETZIGRG5LSESCSH113KLS1M0ITP3VRU14ZTWUT&client_secret=FP00VEYKKEUX01QWVL3O351XDMZCFXZ1CESEPIREXRBME1CN&ll=51.5054,0.0235&v=20181101&query=Gym&radius=2000&limit=30'

In [111]:
results = requests.get(url).json()
# results

> Review the results

In [112]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,50d2063ae4b0f9694a5fbdcc,14 Western Gateway,GB,London,United Kingdom,NaN,363,"[14 Western Gateway, London, Greater London, E...","[{'label': 'display', 'lat': 51.5085764964727,...",51.508576,0.022300,NaN,E16 1BP,Greater London,Westgate Apartments Gym,v-1546736171
1,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,54eb9120498e11557faa55d2,6 mitre passage,GB,Greenwich,United Kingdom,NaN,1299,"[6 mitre passage, Greenwich, Greater London, U...","[{'label': 'display', 'lat': 51.50049531964478...",51.500495,0.006481,NaN,NaN,Greater London,Better Gym,v-1546736171
2,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,50924fdde4b00281a4efd9f1,14 Western Gateway,GB,London,United Kingdom,NaN,333,"[14 Western Gateway, London, Greater London, E...","[{'label': 'display', 'lat': 51.508185, 'lng':...",51.508185,0.021731,NaN,E16 1BN,Greater London,WG's Gym,v-1546736171
3,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,58ce740144587f4e79ecde86,1 Tidal Basin Road,GB,London,United Kingdom,NaN,690,"[1 Tidal Basin Road, London, Greater London, E...","[{'label': 'display', 'lat': 51.508286, 'lng':...",51.508286,0.014679,NaN,E16,Greater London,Hoola Gym,v-1546736171
4,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,55b91353498e0e9252615620,NaN,GB,NaN,United Kingdom,NaN,461,[United Kingdom],"[{'label': 'display', 'lat': 51.50844877766109...",51.508449,0.018989,NaN,NaN,NaN,Capital East Apartments Gym,v-1546736171


In [113]:
dataframe.count()

categories                   26
hasPerk                      26
id                           26
location.address             17
location.cc                  26
location.city                21
location.country             26
location.crossStreet          3
location.distance            26
location.formattedAddress    26
location.labeledLatLngs      26
location.lat                 26
location.lng                 26
location.neighborhood         1
location.postalCode          15
location.state               21
name                         26
referralId                   26
dtype: int64

#### b) Explore the Number of Squash Centres in SOHO *(within a 5km radius)*
> *Represents level of Competition*

In [114]:
search_query2 = 'Squash'
radius2 = 5000
print("Squash within 5 km of SOHO")

Squash within 5 km of SOHO


In [160]:
url_Squash = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query2, radius2, LIMIT)
url_Squash

'https://api.foursquare.com/v2/venues/search?client_id=O1EG5HDYZEETZIGRG5LSESCSH113KLS1M0ITP3VRU14ZTWUT&client_secret=FP00VEYKKEUX01QWVL3O351XDMZCFXZ1CESEPIREXRBME1CN&ll=51.5054,0.0235&v=20181101&query=Squash&radius=5000&limit=50'

In [161]:
results2 = requests.get(url).json()
# assign relevant part of JSON to venues
venues2 = results2['response']['venues']

# tranform venues into a dataframe
Squash_df = json_normalize(venues)
Squash_df.head()

# print('{} Squash Centres were returned by Foursquare.'.format(Squash_df.shape[0]))

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,50d2063ae4b0f9694a5fbdcc,14 Western Gateway,GB,London,United Kingdom,NaN,363,"[14 Western Gateway, London, Greater London, E...","[{'label': 'display', 'lat': 51.5085764964727,...",51.508576,0.022300,NaN,E16 1BP,Greater London,Westgate Apartments Gym,v-1546736171
1,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,54eb9120498e11557faa55d2,6 mitre passage,GB,Greenwich,United Kingdom,NaN,1299,"[6 mitre passage, Greenwich, Greater London, U...","[{'label': 'display', 'lat': 51.50049531964478...",51.500495,0.006481,NaN,NaN,Greater London,Better Gym,v-1546736171
2,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,50924fdde4b00281a4efd9f1,14 Western Gateway,GB,London,United Kingdom,NaN,333,"[14 Western Gateway, London, Greater London, E...","[{'label': 'display', 'lat': 51.508185, 'lng':...",51.508185,0.021731,NaN,E16 1BN,Greater London,WG's Gym,v-1546736171
3,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,58ce740144587f4e79ecde86,1 Tidal Basin Road,GB,London,United Kingdom,NaN,690,"[1 Tidal Basin Road, London, Greater London, E...","[{'label': 'display', 'lat': 51.508286, 'lng':...",51.508286,0.014679,NaN,E16,Greater London,Hoola Gym,v-1546736171
4,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,55b91353498e0e9252615620,NaN,GB,NaN,United Kingdom,NaN,461,[United Kingdom],"[{'label': 'display', 'lat': 51.50844877766109...",51.508449,0.018989,NaN,NaN,NaN,Capital East Apartments Gym,v-1546736171


In [155]:
Squash_df

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,50d2063ae4b0f9694a5fbdcc,14 Western Gateway,GB,London,United Kingdom,NaN,363,"[14 Western Gateway, London, Greater London, E...","[{'label': 'display', 'lat': 51.5085764964727,...",51.508576,0.022300,NaN,E16 1BP,Greater London,Westgate Apartments Gym,v-1546736171
1,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,54eb9120498e11557faa55d2,6 mitre passage,GB,Greenwich,United Kingdom,NaN,1299,"[6 mitre passage, Greenwich, Greater London, U...","[{'label': 'display', 'lat': 51.50049531964478...",51.500495,0.006481,NaN,NaN,Greater London,Better Gym,v-1546736171
2,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,50924fdde4b00281a4efd9f1,14 Western Gateway,GB,London,United Kingdom,NaN,333,"[14 Western Gateway, London, Greater London, E...","[{'label': 'display', 'lat': 51.508185, 'lng':...",51.508185,0.021731,NaN,E16 1BN,Greater London,WG's Gym,v-1546736171
3,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,58ce740144587f4e79ecde86,1 Tidal Basin Road,GB,London,United Kingdom,NaN,690,"[1 Tidal Basin Road, London, Greater London, E...","[{'label': 'display', 'lat': 51.508286, 'lng':...",51.508286,0.014679,NaN,E16,Greater London,Hoola Gym,v-1546736171
4,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,55b91353498e0e9252615620,NaN,GB,NaN,United Kingdom,NaN,461,[United Kingdom],"[{'label': 'display', 'lat': 51.50844877766109...",51.508449,0.018989,NaN,NaN,NaN,Capital East Apartments Gym,v-1546736171
5,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,596b6d149deb7d6f726d423c,NaN,GB,London,United Kingdom,NaN,726,"[London, Greater London, E16 2SB, United Kingdom]","[{'label': 'display', 'lat': 51.501206, 'lng':...",51.501206,0.031537,NaN,E16 2SB,Greater London,Barrier Point Private Gym,v-1546736171
6,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,4df8ed58d4c02ad7341a348e,NaN,GB,East Ham,United Kingdom,NaN,495,"[East Ham, Greater London, United Kingdom]","[{'label': 'display', 'lat': 51.50832060290708...",51.508321,0.018095,NaN,NaN,Greater London,Adriatic Gym,v-1546736171
7,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",False,530651da498ecd98857236a2,Wards Wharf Approach,GB,London,United Kingdom,NaN,1138,"[Wards Wharf Approach, London, Greater London,...","[{'label': 'display', 'lat': 51.50037386680349...",51.500374,0.037809,NaN,E16 2ER,Greater London,Tradewinds Gym,v-1546736171
8,"[{'id': '4bf58dd8d48988d176941735', 'name': 'G...",False,51723bfae4b05b6424893bfd,NaN,GB,NaN,United Kingdom,NaN,1085,[United Kingdom],"[{'label': 'display', 'lat': 51.50243929963339...",51.502439,0.038422,NaN,NaN,NaN,Waterside Park Residents Gym,v-1546736171
9,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",False,4ac518f7f964a5209caf20e3,Caxton Street North,GB,London,United Kingdom,NaN,831,"[Caxton Street North, London, Greater London, ...","[{'label': 'display', 'lat': 51.50950178234238...",51.509502,0.013472,NaN,E16 1JL,Greater London,The Peacock Gymnasium,v-1546736171


## PART 2 - Methodology 
### Linking London Areas to FourSquare Data

#### To identify the ideal area to start up a Squash Centre the following steps were conducted:
1. Calculate the number of gyms within a 2km radius and squash Centres within a 5km radius for each respective area 
2. Count the total number of gyms and squash Centres within the London Area
3. Create a column representing a weighting for 'Demand' and 'Competition' which will be: 
   * Demand = '# of Gyms / Total Number of Gyms'
   * Competition = '# of Squash Centres / Total Number of Squash Centres'
4. Create a last column, "Weighted_Demand" - adding the values from 'demand' and 'competition'
5. Rank the London Areas by Weighted_Demand in descending order

In [128]:
df_London_areas.shape[0]
df_London_areas.head()

,Area,Latitude,Longitude
0,Barking and Dagenham,51.546530,0.125061
1,Barnet,51.605584,-0.207618
2,Bexley,51.459158,0.136218
3,Brent,51.551872,-0.257565
4,Bromley,51.391818,0.026435


#### [Code 1] Creating Code to get Nearby Gyms within 2kms

In [158]:
# Change Limit of Searches of FourSquare
LIMIT = 50

def getNearbyGyms(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION,
            lat,
            lng,
            search_query2, 
            radius2, 
            LIMIT)
#         url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#             CLIENT_ID, 
#             CLIENT_SECRET, 
#             VERSION, 
#             lat, 
#             lng, 
#             radius, 
#             LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Area', 
                  'Area_Latitude', 
                  'Area_Longitude', 
                  'Venue', 
                  'Venue_Latitude', 
                  'Venue_Longitude', 
                  'Venue_Category']
    
    return(nearby_venues)




In [159]:
df_London_Gyms = getNearbyGyms(names=df_London_areas['Area'],
                                   latitudes=df_London_areas['Latitude'],
                                   longitudes=df_London_areas['Longitude']
                                  )

Barking and Dagenham


KeyError: 'groups'

In [153]:
df_London_Gyms.head()

,Area,Area_Latitude,Area_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
0,Barking and Dagenham,51.54653,0.125061,Capital Karts,51.531792,0.118739,Go Kart Track
1,Barking and Dagenham,51.54653,0.125061,Mayesbrook Park,51.549842,0.108544,Park
2,Barking and Dagenham,51.54653,0.125061,Lidl,51.551808,0.113537,Supermarket
3,Barking and Dagenham,51.54653,0.125061,Goodmayes Park,51.556857,0.119759,Park
4,Barking and Dagenham,51.54653,0.125061,Co-op Food,51.540093,0.127522,Grocery Store


In [154]:
df_London_Gyms.groupby('Area').count()

,Area_Latitude,Area_Longitude,Venue,Venue_Latitude,Venue_Longitude,Venue_Category
Area,,,,,,
Barking and Dagenham,30,30,30,30,30,30
Barnet,30,30,30,30,30,30
Bexley,30,30,30,30,30,30
Brent,30,30,30,30,30,30
Bromley,30,30,30,30,30,30
Camden,30,30,30,30,30,30
City of London,30,30,30,30,30,30
Croydon,30,30,30,30,30,30
Ealing,30,30,30,30,30,30


In [152]:
# df_Gyms = df_London_Gyms['Venue_Category'].str.contains("Gym")
# df[        df           ['ids']           .str.contains("ball")]
# df_London_Gyms
# df_London_Gyms


/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match

#### Group by Neighbordhood and Count Number of Unique gyms

#### [Code 2] Creating Code to get Nearby Squash Centres within 5kms

In [124]:
def getNearbySquash(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
df_London_Gyms = getNearbySquash(names=df_London_areas['Neighborhood'],
                                   latitudes=df_London_areas['Latitude'],
                                   longitudes=df_London_areas['Longitude']
                                  )